## Module 5
# Adding and Updating Content

In [ ]:
from arcgis.gis import GIS

# connect to GIS through a profile
gis = GIS(profile='Support')
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.name)

### Explore Content 

#### Using the content.get() fucntion we can find content by the item ID


In [ ]:
# Item Added From Toolbar
# Title: Patrol Information RJC | Type: Feature Service | Owner: ryan_cooper@eagle.co.nz
item = gis.content.get("0b2459b8140c46d28069a8a038a2242a")
item

### Clone Content

Using the clone function will 'Clone' all items provided in the function.
<br/>
You will become the owner of the dataset but all the information will remain the same (including the title)

In [ ]:
# this function will clone the item we defined above
myClone = gis.content.clone_items([item])

# rename the dataset
myClone[0].update(item_properties={'title':'Patrol Information'})

# define the myItem varible that we will use for this Module
myItem = myClone[0]

#### How has this data been shared

In [ ]:
shareStatus = myItem.shared_with

# check if this data has been shared to the Public
if shareStatus['everyone'] == True:
    print ('This data has been shared to the Public')
else:
    print ('This data is not shared to the Public')
    
# check if this data has been shared to your Organisation    
if shareStatus['org'] == True:
    print ('This data has been shared to your Organisation')
else:
    print ('This data is not shared to your Organisation')

# check if this data has been shared to any Groups   
if len(shareStatus['groups']) != 0:    
    for group in shareStatus['groups']:
        print ("This data has been shared to {0}".format(group.title))
else: 
    print ('This data is not shared to any Groups')

#### Change the Sharing Status

In [ ]:
# do not share this publicaly
myItem.share(everyone=True, org=True)
# re-run the above cell to test this has worked

### Inspect the data within our Item

In [ ]:
# pull out the desired layer of the feature service
layer = myItem.layers[0]
print (layer)

In [ ]:
# get the properties of the layer
print (layer.properties.extent.spatialReference)

In [ ]:
# query the layer - I want to return all layers
featureSet = layer.query()
print (featureSet)

In [ ]:
# explore the features within the feature set
features = featureSet.features
print (features[0])

### Read in Weekly Surf Reports

In [ ]:
import pandas as pd

In [ ]:
surfReport_CSV = r'https://github.com/RJC32/WorkshopIcons/blob/master/Users/Week0.csv?raw=true'

surfReport = pd.read_csv(surfReport_CSV)

# Using both Pandas and Jupyter Notebooks we can print the sheet
surfReport.head()

### Update a Data in a Feature Service

In [ ]:
# create an empty list that will contain the info for all our features that we are going to update
myFeatures = []

for f in features:
    for index, row in surfReport.iterrows():
        if f.attributes['OBJECTID'] == (row['ID']):
            
            # create empty dictionaries that we will store our Attributes and Geometry in.
            myAttributes = {}
            myGeometry = {}
            
            # confirm the object id
            myAttributes['objectid'] = f.attributes['OBJECTID']
            
            # confirm the location data
            myGeometry['x'] = f.geometry['x']
            myGeometry['y'] = f.geometry['y']

            # pull out the data for each location
            myAttributes['beach'] = f.attributes['Beach']
            myAttributes['patrol'] = f.attributes['Patrol']
            myAttributes['vistors'] = f.attributes['Vistors'] + (row['Vistors'])
            myAttributes['rescues'] = f.attributes['Rescues'] + (row['Rescues'])
            myAttributes['minor'] = f.attributes['Minor'] + (row['Minor'])
            myAttributes['major'] = f.attributes['Major'] + (row['Major'])
            myAttributes['cpr'] = f.attributes['CPR'] + (row['CPR'])
            myAttributes['casualties'] = f.attributes['Casualties'] + (row['Casualties'])
            
            newFeature = {"geometry": myGeometry, "attributes": myAttributes}
            myFeatures.append(newFeature)
            
layer.edit_features(updates=myFeatures)  